In [1]:
import numpy as np
import pandas as pd

In [7]:
# Here we load the dataset "data.csv" generated in  in "cdp-notebook.ipynb"
# It is a concatenation of eeach yearly report with a mapping of columns

data = pd.read_csv("../../../data/ghg-emissions/cdp/cdp_data_all_years.csv")

In [8]:
data.head()

,Organization,Account Number,Country,City,C40,Reporting Year,Accounting year,Primary Protocol,Primary Protocol Comment,Total City-wide Emissions,...,Land area (in square km),Population,Population Year,Average altitude (m),Average annual temperature (in Celsius),City GDP,GDP Currency,Year of GDP,GDP Source,Last update
0,Kaohsiung City Government,31112,Taiwan,Kaohsiung,NaN,2012,2010,2006 IPCC Guidelines for National Greenhouse G...,The methodology we applied is based on 2006 IP...,63624500.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Taipei City Government,31446,Taiwan,Taipei,NaN,2012,2010,Other: IPCC 1995 GWP,GHG emissions counting in Taipei City contain ...,15500000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Greater London Authority,3422,United Kingdom,London,C40,2012,2010,Other: Defra's 2009 guidance on how to measure...,The London Energy and Greenhouse Gas Inventory...,43400000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City of Melbourne,31109,Australia,Melbourne,C40,2012,2009,NaN,Melbourne's methodology and boundary for munic...,4870289.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bogotá Distrito Capital,31154,Colombia,Bogotá,C40,2012,2011,International Standard for Determining Greenho...,"For Greenhouse Gas Inventory in Bogotá, we use...",15921690.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# As for the time component, shouldn't we use a "start_date" and "end_date" ?
# This column needs preprocessing because they are not all in the same formatting
data["Accounting year"].value_counts()

2016-01-01 - 2016-12-31    307
2018-01-01 - 2018-12-31    268
2017-01-01 - 2017-12-31    267
2015-01-01 - 2015-12-31    237
2014-01-01 - 2014-12-31    194
                          ... 
2018-11-01 - 2019-05-31      1
2006-01-01 - 2006-12-31      1
2018-01-01 - 2018-12-30      1
2019-08-01 - 2020-08-01      1
2015-05-01 - 2016-05-01      1
Name: Accounting year, Length: 170, dtype: int64